### Experiments with the AlphaVantage API

In [1]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
from datetime import datetime, timedelta

In [2]:
load_dotenv()

True

In [3]:
ALPHA_VANTAGE_API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")

In [4]:
# Crypto exchange rates could be a fun project (or an extension to this project) too!
# # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
# url = f'https://www.alphavantage.co/query?function=CURRENCY_EXCHANGE_RATE&from_currency=BTC&to_currency=EUR&apikey={ALPHA_VANTAGE_API_KEY}'
# r = requests.get(url)
# data = r.json()

# print(data)

### Fetch 100 ticker data points at a specified interval (in minutes; takes only 1, 5, 15, 30 and 60)

In [5]:
def fetch_ticker_data(symbol: str, interval: int, apiKey: str) -> pd.DataFrame:
    base_url = f'https://www.alphavantage.co/query'
    interval = str(interval) + "min"
    params = {
        "function": "TIME_SERIES_INTRADAY",
        "symbol": symbol,
        "interval": interval,
        "apikey": apiKey,
        "outputsize": "compact",
    }
    try:
        response = requests.get(base_url, params=params)
        data = response.json()[f"Time Series ({interval})"]
        df = pd.DataFrame(data)
        df = df.transpose()
        df = df.reset_index().rename(columns={"index": "timestamp", "1. open": "open", "2. high": "high", "3. low": "low", "4. close": "close", "5. volume": "volume"})
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        return df
    except Exception as e:
        print(e)
        return None

In [6]:
df = fetch_ticker_data(symbol="AAPL", interval=5, apiKey=ALPHA_VANTAGE_API_KEY)

In [7]:
df

,timestamp,open,high,low,close,volume
0,2025-01-23 19:55:00,223.2300,223.2500,223.2000,223.2200,6588
1,2025-01-23 19:50:00,223.2201,223.3000,223.2000,223.2300,6412
2,2025-01-23 19:45:00,223.2000,223.2500,223.2000,223.2207,2582
3,2025-01-23 19:40:00,223.1900,223.2500,223.1500,223.2000,2058
4,2025-01-23 19:35:00,223.1900,223.2000,223.1500,223.1800,3568
...,...,...,...,...,...,...
95,2025-01-23 12:00:00,224.0600,224.1600,223.3857,223.4200,488965
96,2025-01-23 11:55:00,223.9200,224.1950,223.8500,224.0400,370532
97,2025-01-23 11:50:00,224.3200,224.3705,223.8000,223.9200,580552
98,2025-01-23 11:45:00,224.4500,224.5900,224.2500,224.3200,356223


### Filter by time

In [8]:
# # est_time = datetime(2024, 1, 23, 14, 5, 0).replace(minute=0, second=0, microsecond=0) + timedelta(hours=3)
# est_time = datetime.now().replace(minute=0, second=0, microsecond=0) + timedelta(hours=3)
# print(type(est_time))
# print(est_time)

In [9]:
# df[df["timestamp"]==est_time]

In [10]:
def filter_stock_data(df: pd.DataFrame) -> pd.DataFrame:
    now = datetime.now()
    minutes = now.minute
    round_down_minutes = minutes // 5 * 5
    pst_time = datetime.now().replace(minute=round_down_minutes, second=0, microsecond=0) + timedelta(hours=3) # Rounds down time to the closest 5n minutes
    return_df = df[df["timestamp"]==pst_time]
    if return_df.empty:
        return None
    return return_df

In [11]:
print(filter_stock_data(df))

None
